# Prompt Notebook with Chat - Prompt Lab Notebook v1.1.0
This notebook contains steps and code to demonstrate inferencing of prompts
generated in Prompt Lab in watsonx.ai with a chat format. It introduces Python API commands
for authentication using API key and prompt inferencing using WML API.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Defining parameters of the Model object
* Using the Model object to generate response using the defined model id, parameters and the prompt input

# Setup

In [1]:
!pip install --upgrade 'chromadb==0.3.26' 'pydantic==1.10.0' sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 4.6 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 45.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 972.9/972.9 kB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 49.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 64.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 66.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 66.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 63.6 MB/s eta 0:00:00

  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp310-cp310-linux_x86_64.whl size=187971 sha256=b888c0f22ae7ebc08417ee50e8d457857d5e58082e623c6ec228d5d5a55618f6
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/af/a9/3e/3e5d59ee41664eb31a4e6de67d1846f86d16d93c45f277c4e7
Successfully built hnswlib
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0


## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [2]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://eu-de.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("JM59wATYpa21z8IWGcTPwlwFFR39xZwGSNNASVPuCCdA")
	}


# Inferencing
This cell demonstrated how we can use the model object as well as the created access token
to pair it with parameters and input string to obtain
the response from the the selected foundation model.

## Defining the model id
We need to specify model id that will be used for inferencing:


In [3]:
model_id = "sdaia/allam-1-13b-instruct"


## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [4]:
parameters = {
    "decoding_method": "greedy",
    "max_new_tokens": 800,
    "repetition_penalty": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [5]:
project_id = os.getenv("PROJECT_ID")#a9606b37-4f21-492c-90f2-ac1873e40946
space_id = os.getenv("SPACE_ID")


## Defining the Model object
We need to define the Model object using the properties we defined so far:


In [6]:
from ibm_watsonx_ai.foundation_models import Model

model = Model(
	model_id = model_id,
	params = parameters,
	credentials = get_credentials(),
	project_id = project_id,
	space_id = space_id
	)


JM59wATYpa21z8IWGcTPwlwFFR39xZwGSNNASVPuCCdA········


## Defining the vector index
Initialize the vector index to query when chatting with the model.

In [7]:
from ibm_watsonx_ai.client import APIClient

wml_credentials = get_credentials()
client = APIClient(credentials=wml_credentials, project_id=project_id, space_id=space_id)

vector_index_id = "1fb02077-a0b8-4e22-b1c7-3e3a931a0fd3"
vector_index_details = client.data_assets.get_details(vector_index_id)
vector_index_properties = vector_index_details["entity"]["vector_index"]

from ibm_watsonx_ai.foundation_models.embeddings.sentence_transformer_embeddings import SentenceTransformerEmbeddings

emb = SentenceTransformerEmbeddings('sentence-transformers/all-MiniLM-L6-v2')

import subprocess
import gzip
import json
import chromadb
import random
import string

def hydrate_chromadb():
    data = client.data_assets.get_content(vector_index_id)
    content = gzip.decompress(data)
    stringified_vectors = str(content, "utf-8")
    vectors = json.loads(stringified_vectors)

    chroma_client = chromadb.Client()

    # make sure collection is empty if it already existed
    collection_name = "my_collection"
    try:
        collection = chroma_client.delete_collection(name=collection_name)
    except:
        print("Collection didn't exist - nothing to do.")
    collection = chroma_client.create_collection(name=collection_name)

    vector_embeddings = []
    vector_documents = []
    vector_metadatas = []
    vector_ids = []

    for vector in vectors:
        vector_embeddings.append(vector["embedding"])
        vector_documents.append(vector["content"])
        metadata = vector["metadata"]
        lines = metadata["loc"]["lines"]
        clean_metadata = {}
        clean_metadata["asset_id"] = metadata["asset_id"]
        clean_metadata["asset_name"] = metadata["asset_name"]
        clean_metadata["url"] = metadata["url"]
        clean_metadata["from"] = lines["from"]
        clean_metadata["to"] = lines["to"]
        vector_metadatas.append(clean_metadata)
        asset_id = vector["metadata"]["asset_id"]
        random_string = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))
        id = "{}:{}-{}-{}".format(asset_id, lines["from"], lines["to"], random_string)
        vector_ids.append(id)

    collection.add(
        embeddings=vector_embeddings,
        documents=vector_documents,
        metadatas=vector_metadatas,
        ids=vector_ids
    )
    return collection

chroma_collection = hydrate_chromadb()

def proximity_search( question ):
    query_vectors = emb.embed_query(question)
    query_result = chroma_collection.query(
        query_embeddings=query_vectors,
        n_results=vector_index_properties["settings"]["top_k"],
        include=["documents", "metadatas", "distances"]
    )

    documents = list(reversed(query_result["documents"][0]))

    return "\n".join(documents)


JM59wATYpa21z8IWGcTPwlwFFR39xZwGSNNASVPuCCdA········


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Collection didn't exist - nothing to do.


## Defining the inferencing input for chat
Foundation models supporting chat accept a system prompt that instructs the model on how to conduct the dialog. They also accept previous questions and answers to give additional context when inferencing. Each model has it's own string format for constructing the input.

Let us provide the input we got from the Prompt Lab and format it for the selected model:


In [8]:
prompt_input = """
__grounding__

<<SYS>>
لنلعب لعبة بينجو أحرف الجر لتعليم قواعد اللغة العربية. هذه اللعبة مصممة لجميع المستويات: المبتدئ، والمتوسط، والمتقدم.
أهداف اللعبة: تدريب اللاعبين على استخدام حروف الجر بشكل صحيح في الجمل.
مكونات اللعبة:
 بطاقة بينجو: جدول يحتوي على خلايا فارغة لكتابة الجمل. عدد الخلايا يختلف حسب المستوى ولكن يجب أن تساوي عدد الأسطر عدد الأعمدة.
أحرف الجر: مجموعة من أحرف الجر التي يتعلمها اللاعب حسب مستواه.
جمل: مجموعة متنوعة من الجمل المكتملة فيها فراغات لحروف الجر، تزيد عدد الفراغات مع صعوبة المستوى ولكنها فقط فراغات لحروف الجر.
قواعد اللعبة:
في بداية اللعبة، يتم شرح مفهوم أحرف الجر، تحديد الحروف المدخلة في اللعبة، وأمثلة عليها.
يتم توليد جملة يجب أن تحتوي فراغاً على الأقل مكان حرف الجر واللاعب يختار ويختار رقم الخلية.
الدخل: يكتب اللاعب حرف الجر الصحيح ورقم الخلية الفارغة لحفظ الجملة فيها. إذا كانت الخلية مستخدمة مسبقاً عليه أن يختار غيرها. وإذا كانت الإجابة خاطئة يلغى الحقل المختار بحيق لايمكن رسم خطوط منه,
يكمل اللاعب صفًا أو عمودًا أو قطراً كاملاً من الخلايا المعبأة بالجمل التي تحوي إجابات صحيحة ليفوز.
مستويات اللعبة:
المستوى المبتدئ: يركز على أحرف الجر الأساسية (من، إلى، عن، على، في). عدد خلايا البطاقة 4.
المستوى المتوسط: يضيف أحرف جر جديدة (الباء، اللام، الكاف). عدد خلايا البطاقة 6.
المستوى المتقدم: يشتمل على أحرف جر أكثر تعقيدًا (واو القسم، تاء القسم، مذ، منذ، ربما، حتى). عدد خلايا البطاقة 9.
آلية اللعب:
يتم تقديم جمل متنوعة لتغطية مختلف المواقف اللغوية.
تزداد صعوبة الجمل تدريجيًا مع تقدم اللعبة.
يتم تقديم نصائح وإرشادات للاعبين لمساعدتهم على فهم القواعد.
مثال على الجملة: ذهبت ... المدرسة. (اللاعب يكتب "إلى" ويكتب رقم أحد الخلايا لتحفظ الجملة في الخلية المختارة.)

<</SYS>>
"""


## Execution
Let us now use the defined Model object, pair it with the input, and generate the response to your question:


In [ ]:
user_input = input("هل أنت جاهز للعب: ")
while user_input != "توقف":
    grounding = proximity_search(user_input)
    formattedQuestion = f"""<s> [INST] {user_input} [/INST]"""
    prompt = f"""{prompt_input}{formattedQuestion}"""
    generated_response = model.generate_text(prompt=prompt.replace("__grounding__", grounding), guardrails=False)
    print(f"AI: {generated_response}")
    user_input = input("إجابتك: ")


هل أنت جاهز للعب: جاهز
AI:  شكراً لاهتمامك باللعبة! سأشرح لك كيفية لعب بينجو أحرف الجر باللغة العربية. سنستخدم مستوى متوسط كمثال للتوضيح:

1. اختر بطاقة بينجو تحتوي على 6 أعمدة و6 صفوف (36 خلية).

2. حدد أحرف الجر التي ستتعلمها في هذا المستوى: الباء، اللام، الكاف.

3. اشرح مفهوم أحرف الجر وأمثلة عليها للاعبين.

4. قم بتوليد جمل متنوعة تحتوي على فراغات مكان أحرف الجر. على سبيل المثال:
   - اشتريت ... بـ... من المتجر.
   - سافرت ... لـ... لزيارة أقاربي.
   - قرأت ... كـ... من الكتاب.

5. يختار كل لاعب بطاقة بينجو ويبدأ بملء الخلايا بالجمل المناسبة.

6. عندما يجد اللاعب جملة تحتوي على فراغ لحرف الجر، يختار حرف الجر المناسب ويكتب رقم الخلية لحفظ الجملة.

7. إذا كان اللاعب قد استخدم حرف الجر في خلية أخرى، يجب عليه اختيار خلية أخرى للجملة الحالية.

8. إذا كانت الإجابة خاطئة، يُلغى الحقل المختار بحيث لا يمكن رسم خطوط منه.

9. يكمل اللاعبون الصفوف أو الأعمدة أو القطرات الكاملة من الخلايا المعبأة بالجمل الصحيحة للفوز.

10. يمكن تقديم نصائح وإرشادات للاعبين لمساعدتهم على فهم القواعد وتحسين استخد

# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2023 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  